In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm
import numpy as np

def load_model_and_tokenizer():
    model_name = "gpt4o-mini"  # Replace with the actual model name/path
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return model, tokenizer

def load_helpsteer_dataset():
    dataset = load_dataset("nvidia/HelpSteer")
    return dataset["test"]  # Assuming we're using the test split for evaluation

def evaluate_model(model, tokenizer, dataset):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    total_loss = 0
    num_samples = 0

    for sample in tqdm(dataset):
        input_text = sample["input"]
        target_text = sample["target"]

        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)
        targets = tokenizer(target_text, return_tensors="pt", truncation=True, max_length=512).to(device)

        with torch.no_grad():
            outputs = model(**inputs, labels=targets["input_ids"])
            loss = outputs.loss

        total_loss += loss.item()
        num_samples += 1

    average_loss = total_loss / num_samples
    perplexity = np.exp(average_loss)

    return average_loss, perplexity

def main():
    print("Loading model and tokenizer...")
    model, tokenizer = load_model_and_tokenizer()

    print("Loading HelpSteer dataset...")
    dataset = load_helpsteer_dataset()

    print("Evaluating model...")
    avg_loss, perplexity = evaluate_model(model, tokenizer, dataset)

    print(f"Evaluation results:")
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Perplexity: {perplexity:.4f}")

In [2]:
main()

Loading model and tokenizer...


OSError: gpt4o-mini is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.